# Coordinate system creation

This example shows how to use PyAEDT to create and modify coordinate systems in the modeler.

Keywords: **AEDT**, **modeler**, **coordinate system**.

## Perform imports and define constants
Import the required packages.

In [1]:
import os
import tempfile
import time

import ansys.aedt.core


Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False  # Open the AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Launch AEDT

In [4]:
d = ansys.aedt.core.launch_desktop(version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=True)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_33838d31-cc84-48af-bb9f-dcbcd9a9b104.log is enabled.


...

PyAEDT INFO: New AEDT gRPC session session started on port 50051.


PyAEDT INFO: 2025.2 version started with process ID 8276.


PyAEDT WARNING: Service Pack is not detected. PyAEDT is currently connecting in Insecure Mode.


PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.


PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


## Insert HFSS design

Insert an HFSS design with the default name.

In [5]:
project_name = os.path.join(temp_folder.name, "CoordSysDemo.aedt")
hfss = ansys.aedt.core.Hfss(version=AEDT_VERSION, project=project_name)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8276!


PyAEDT INFO: Project CoordSysDemo has been created.


PyAEDT INFO: No design is present. Inserting a new design.


PyAEDT INFO: Added design 'HFSS_FGZ' of type HFSS.


PyAEDT INFO: Aedt Objects correctly read


## Create coordinate system

The coordinate system is centered on the global origin and has the axis
aligned to the global coordinate system. The new coordinate system is
saved in the object ``cs1``.

In [6]:
cs1 = hfss.modeler.create_coordinate_system()

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


## Modify coordinate system

The ``cs1`` object exposes properties and methods to manipulate the
coordinate system. The origin can be changed.

In [7]:
cs1["OriginX"] = 10
cs1.props["OriginY"] = 10
cs1.props["OriginZ"] = 10

The orientation of the coordinate system can be modified by
updating the direction vectors for the coordinate system.

In [8]:
ypoint = [0, -1, 0]
cs1.props["YAxisXvec"] = ypoint[0]
cs1.props["YAxisYvec"] = ypoint[1]
cs1.props["YAxisZvec"] = ypoint[2]

## Rename coordinate system

Rename the coordinate system.

In [9]:
cs1.rename("newCS")

True

## Change coordinate system mode

Use the ``change_cs_mode`` method to change the mode. Options are:

- ``0`` for axis/position
- ``1`` for Euler angle ZXZ
- ``2`` for Euler angle ZYZ

Here ``1`` sets Euler angle ZXZ as the mode.

In [10]:
cs1.change_cs_mode(1)

True

The following lines use the ZXZ Euler angle definition to rotate the coordinate system.

In [11]:
cs1.props["Phi"] = "10deg"
cs1.props["Theta"] = "22deg"
cs1.props["Psi"] = "30deg"

## Delete coordinate system

Delete the coordinate system.

In [12]:
cs1.delete()

True

## Define a new coordinate system

Create a coordinate system by defining the axes. You can
specify all coordinate system properties as shown here.

In [13]:
cs2 = hfss.modeler.create_coordinate_system(
    name="CS2",
    origin=[1, 2, 3.5],
    mode="axis",
    x_pointing=[1, 0, 1],
    y_pointing=[0, -1, 0],
)

A new coordinate system can also be created based on the Euler angle convention.

In [14]:
cs3 = hfss.modeler.create_coordinate_system(name="CS3", origin=[2, 2, 2], mode="zyz", phi=10, theta=20, psi=30)

Create a coordinate system that is defined by standard views in the modeler. The options are:

- ``"iso"``
- ``"XY"``
- ``"XZ"``
- ``"XY"``

Here ``"iso"`` is specified. The axes are set automatically.

In [15]:
cs4 = hfss.modeler.create_coordinate_system(name="CS4", origin=[1, 0, 0], reference_cs="CS3", mode="view", view="iso")

## Create coordinate system by defining axis and angle rotation

Create a coordinate system by defining the axis and angle rotation. When you
specify the axis and angle rotation, this data is automatically translated
to Euler angles.

In [16]:
cs5 = hfss.modeler.create_coordinate_system(name="CS5", mode="axisrotation", u=[1, 0, 0], theta=123)

Face coordinate systems are bound to an object face.
First create a box and then define the face coordinate system on one of its
faces. To create the reference face for the face coordinate system, you must
specify starting and ending points for the axis.

In [17]:
box = hfss.modeler.create_box([0, 0, 0], [2, 2, 2])
face = box.faces[0]
fcs1 = hfss.modeler.create_face_coordinate_system(face=face, origin=face.edges[0], axis_position=face.edges[1], name="FCS1")

PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


Create a face coordinate system centered on the face with the X axis pointing
to the edge vertex.

In [18]:
fcs2 = hfss.modeler.create_face_coordinate_system(face=face, origin=face, axis_position=face.edges[0].vertices[0], name="FCS2")

Swap the X axis and Y axis of the face coordinate system. The X axis is the
pointing ``axis_position`` by default. You can optionally select the Y axis.

In [19]:
fcs3 = hfss.modeler.create_face_coordinate_system(face=face, origin=face, axis_position=face.edges[0], axis="Y")

The face coordinate system can also be rotated by changing the
reference axis.

In [20]:
fcs3.props["WhichAxis"] = "X"

### Rotate the coordinate system

Apply a rotation around the Z axis. The Z axis of a face coordinate system
is always orthogonal to the face. A rotation can be applied at definition.
Rotation is expressed in degrees.

In [21]:
fcs4 = hfss.modeler.create_face_coordinate_system(face=face, origin=face, axis_position=face.edges[1], rotation=10.3)

Rotation can also be changed after coordinate system creation.

In [22]:
fcs4.props["ZRotationAngle"] = "3deg"

### Offset the coordinate system

Apply an offset to the X axis and Y axis of a face coordinate system.
The offset is in respect to the face coordinate system itself.

In [23]:
fcs5 = hfss.modeler.create_face_coordinate_system(face=face, origin=face, axis_position=face.edges[2], offset=[0.5, 0.3])

The offset can be changed after the coordinate system has been created.

In [24]:
fcs5.props["XOffset"] = "0.2mm"
fcs5.props["YOffset"] = "0.1mm"

### Create a dependent coordinate system

The use of dependent coordinate systems can simplify model creation. The following
cell shows how to create a coordinate system whose reference is the face coordinate system.

In [25]:
face = box.faces[1]
fcs6 = hfss.modeler.create_face_coordinate_system(face=face, origin=face, axis_position=face.edges[0])
cs_fcs = hfss.modeler.create_coordinate_system(name="CS_FCS", origin=[0, 0, 0], reference_cs=fcs6.name, mode="view", view="iso")

### Create object coordinate systems

A coordinate system can also be defined relative to elements
belonging to an object. For example, the coordinate system can be
connected to an object face.

In [26]:
obj_cs = hfss.modeler.create_object_coordinate_system(
    assignment=box,
    origin=box.faces[0],
    x_axis=box.edges[0],
    y_axis=[0, 0, 0],
    name="box_obj_cs",
)
obj_cs.rename("new_obj_cs")

True

Create an object coordinate system whose origin is linked to the edge of an object.

In [27]:
obj_cs_1 = hfss.modeler.create_object_coordinate_system(
    assignment=box.name,
    origin=box.edges[0],
    x_axis=[1, 0, 0],
    y_axis=[0, 1, 0],
    name="obj_cs_1",
)
obj_cs_1.set_as_working_cs()

True

Create an object coordinate system with an origin specified on a point within an object.

In [28]:
obj_cs_2 = hfss.modeler.create_object_coordinate_system(
    assignment=box.name,
    origin=[0, 0.8, 0],
    x_axis=[1, 0, 0],
    y_axis=[0, 1, 0],
    name="obj_cs_2",
)
new_obj_cs_2 = hfss.modeler.duplicate_coordinate_system_to_global(obj_cs_2)
obj_cs_2.delete()

True

Create an object coordinate system with an origin on the vertex.

In [29]:
obj_cs_3 = hfss.modeler.create_object_coordinate_system(
    assignment=box.name,
    origin=box.vertices[1],
    x_axis=box.faces[2],
    y_axis=box.faces[4],
    name="obj_cs_3",
)
obj_cs_3.props["MoveToEnd"] = False
obj_cs_3.update()

True

### Get all coordinate systems

Easily retrieve and subsequently manipulate all coordinate systems.

In [30]:
css = hfss.modeler.coordinate_systems
names = [i.name for i in css]
print(names)

['obj_cs_3', 'obj_cs_1', 'new_obj_cs', 'CS_FCS', 'Face_CS_HIGPAL', 'Face_CS_GI3E1L', 'Face_CS_SNOZYG', 'Face_CS_R2O6V2', 'FCS2', 'FCS1', 'CS5', 'CS4', 'CS3', 'CS2']


## Select coordinate system

Select an existing coordinate system.

In [31]:
css = hfss.modeler.coordinate_systems
cs_selected = css[0]
cs_selected.delete()

True

## Get point coordinate under another coordinate system

Get a point coordinate under another coordinate system. A point coordinate
can be translated in respect to any coordinate system.

In [32]:
hfss.modeler.create_box([-10, -10, -10], [20, 20, 20], "Box1")
p = hfss.modeler["Box1"].faces[0].vertices[0].position
print("Global: ", p)
p2 = hfss.modeler.global_to_cs(p, "CS5")
print("CS5 :", p2)

Global:  [-10.0, -10.0, 10.0]
CS5 : [-0.7074753504689, 8.5404490111998, -15.0519171309031]


## Release AEDT
Close the project and release AEDT.

In [33]:
d.release_desktop()
time.sleep(3)

PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``. If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell removes all temporary files, including the project folder.

In [34]:
temp_folder.cleanup()